In [5]:
import nltk
nltk.download('punkt')
import re
from gensim.models import Word2Vec
import gensim.downloader as api

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##Clean corpus

In [38]:
# with open('Output_0.txt', 'r', encoding='utf-8-sig') as f:
#     output_0 = f.read()

# with open('Output_1.txt', 'r', encoding='utf-8-sig') as f:
#     output_1 = f.read()

# with open('Output_2.txt', 'r', encoding='utf-8-sig') as f:
#     output_2 = f.read()

# text = " ".join([output_0,output_1,output_2])

In [43]:
with open('pride_and_prejudice.txt', 'r', encoding='utf-8-sig') as f:
    text = f.read()
sentences = nltk.sent_tokenize(text)

# Tokenize each sentence into words
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

# Convert all words to lowercase and remove any leading/trailing white spaces
novel = [[word.lower().strip() for word in sentence] for sentence in tokenized_sentences]

# define a regular expression to match non-alphabetic characters
pattern = re.compile('[^a-z]+')

def cleanWord(word):
    clean_word = re.sub(r'[^a-zA-Z]', '', word)
    return clean_word.lower() # Convert to lowercase and return

# remove non-alphabetic characters and convert to lowercase
clean_novel = []
for sentence in novel:
    clean_sentence = []
    for word in sentence:
        if bool(re.search('[a-zA-Z]', word)) and bool(re.search('[^a-z]+', word)): # if word contains alphabets & non-alphabetic characters
            clean_word = pattern.sub(' ', word).split() # substitute non-alphabetic characters with space
            for word in clean_word:
                clean_sentence.append(word)
        else:
            clean_sentence.append(word)
    clean_novel.append(clean_sentence)

## Continue train embeddings

In [44]:
glove_model = api.load('glove-wiki-gigaword-50')

new_model = Word2Vec(vector_size=50, window=20, min_count=1, workers=4)
new_model.build_vocab(clean_novel)

for key in new_model.wv.index_to_key:
    if key in glove_model:
        new_model.wv[key] = glove_model[key]
    elif cleanWord(key) in glove_model:
        new_model.wv[cleanWord(key)] = glove_model[cleanWord(key)]

new_model.train(novel,total_examples=new_model.corpus_count, epochs=new_model.epochs)

(509554, 772195)

In [46]:
new_model.save("pp.model")